In [1]:
# To load the Pandas libraries as an alias 'pds' 
import pandas as pds
# To load the numpy libraries as an alias 'npy' 
import numpy as npy 

# Importing Label Encoder for encoding categorical feature
from sklearn.preprocessing import LabelEncoder
import csv 

#loading data
Fish = pds.read_csv("Fish.csv")

#Splitting training features and labels
train_features = Fish.drop(columns = 'Species',axis=1).values
train_labels =  Fish['Species'].values

FishData = pds.DataFrame(train_features)
x = npy.array([train_features], npy.int32)

#To get n number of datapints to be selected as the medoids
Init_Medoids = npy.array(FishData.sample(n=6, frac=None, replace=False, weights=None, random_state=None, axis=None))

#Function to Calculate Manhattan Distance
def Calc_Manhattan_Dist(TestData, TrainData):
    dist = 0
    for i in range(TrainData.shape[0]):
        dist = dist + abs(TrainData[i] - TestData[i])
    return dist

p1 = npy.array(x[0,:], npy.int32)
p2 = Init_Medoids.astype(int)

#Clustering process will continue until cluster assignments do not change or maximum 100 iterations have been completed
for i in range(0,100):    

    final=[]
    for x in p1:
        final.append([Calc_Manhattan_Dist(x,j) for j in p2])
    # print(final)

    #Assigning labels to the data points in the cluster that they are assigned
    def label_assignment(final):
        labels = npy.argmin(final, axis = 1)
        return labels

    labels = label_assignment(final)

    labels_df = pds.DataFrame(labels) 

    dictionary_in={}
    for row in labels_df.index:
        if labels_df.iloc[row,0] not in dictionary_in.keys():
            dictionary_in[labels_df.iloc[row,0]]=[row]
        else:
            dictionary_in[labels_df.iloc[row,0]].append(row)
#     print (dictionary_in)

    #Calculating new cluster heads by taking distance of each data point to data points to clusters from which 
    # it does'nt belong and assigning head to a cluster which has minimum distance
    new_heads = []    
    for key,item in dictionary_in.items():
        all_point_cost=[]
        for j in item:
            results=0
            for k in item:
                results += Calc_Manhattan_Dist(p1[j], p1[k])
            all_point_cost.append(results)
        #lsit of cost for all points find min and repalce with mediod
        index = item[npy.argmin(all_point_cost)]

#         print(index)
        new_heads.append(p1[index,:])

    new_heads
    x_new = npy.array([train_features], npy.int32)

    p1_new = npy.array(x_new[0,:], npy.int32)
    p2_new = npy.array(new_heads, npy.int32)

    p2=p2_new

    final_new = []
    for x in p1_new:
        final_new.append([Calc_Manhattan_Dist(x,j) for j in p2_new])
#     print(final_new)

    #CLuster Assignments
    
    labels_new = label_assignment(final_new)
    labels_df_new = pds.DataFrame(labels_new) 
    
    dictionary_in_new={}
    for row in labels_df_new.index:
        if labels_df_new.iloc[row,0] not in dictionary_in_new.keys():
            dictionary_in_new[labels_df_new.iloc[row,0]]=[row]
        else:
            dictionary_in_new[labels_df_new.iloc[row,0]].append(row)
    print (dictionary_in_new)    
    
    if {k: dictionary_in[k] for k in dictionary_in if k in dictionary_in_new and dictionary_in[k] == dictionary_in_new[k]}: 
        break
    else:
        i +=1

dic_grp={}       
for key1,value1 in dictionary_in_new.items():
    for values in value1:
        dic_grp[values]=key1
# print(dic_grp)

#Appending cluster_id i.e cluster to which that data point belongs to excel
df_grp = pds.DataFrame.from_dict(dic_grp, orient='index')
df_grp.columns = ['Cluster_ID']
# print(df_grp)

resulting_df = Fish.join(df_grp)
# print(resulting_df)

resulting_df.to_csv(r'Fish.csv')


{0: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 70, 71], 1: [14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 58, 59, 60, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 139, 140, 141, 142, 143, 144], 2: [35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47], 3: [61, 62, 63, 64, 65, 66, 67, 68, 69, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91], 4: [92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138], 5: [145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158]}
{0: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 18, 19, 20, 22, 54], 1: [16, 17, 21, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 58, 59, 60, 111, 113, 114, 116, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 139, 140, 141, 142, 143, 144], 2: [35, 36, 37, 38, 39, 40,

In [2]:
#calculating a for silhouette width
#for each feature finding average distance between the point and all other members of the same cluster
members = 0
dicp={}
for key,item in dictionary_in_new.items():
    listp = []
    ll=[]
    for j in item:
        results=0
        for k in item:
            results =Calc_Manhattan_Dist(p1[j], p1[k])
            #print(results)
            listp.append(results)
        ll.append(sum(listp)/len(listp))
    dicp[key]=ll
# print(dicp)

#calculating b for silhouette width
# for nearest cluster calculating average distance between the point and each cluster to which it does not belong
# and minimum of all distances calculated and applying formula to calculate silhouette width

dic_S={}
for key1,value1 in dictionary_in_new.items():
    kl=[]
    for key2,value2 in dictionary_in_new.items():
        
        if key1 != key2:
            ll=[]
            
            for j in value1:
                listp = []
                
                results = 0
                for k in value2:
                    results = Calc_Manhattan_Dist(p1[j], p1[k])
                    listp.append(results)
                ll.append(sum(listp)/len(listp))
            
            kl.append(ll)
            
    
    res = [min(idx) for idx in zip(*kl)] 
    
    dic_S[key1]=res

#Calculating silhouette width for each cluster and average silhouette width
listjh=[]
dic_wid={}
for (key1,value1), (key2,value2) in zip(dicp.items(), dic_S.items()):
    for i,j in zip(value1,value2):
        S=(j-i)/max(i,j)
        listjh.append(S)
    avg_wid=(sum(listjh)/len(listjh)) 
    dic_wid[key1]=avg_wid
print("Cluster: Silhouette Width\n",dic_wid)

print("Average Silhouette Width:")
keycount = 0
for key,value in dic_wid.items():
    keycount += 1
    AvgSW = (sum(dic_wid.values()))/keycount
print(AvgSW)


Cluster: Silhouette Width
 {0: 0.5906659387109952, 1: 0.22640885525166438, 2: 0.29932536088702194, 3: 0.34978501081135227, 4: 0.3235722571351998, 5: 0.37761974906153956}
Average Silhouette Width:
0.36122952864296215
